In [2]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd 

In [2]:
import pickle

In [3]:
import seaborn as sns 
import matplotlib.pyplot as plt 

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/experiment-tracking/mlruns/1', creation_time=1716313186845, experiment_id='1', last_update_time=1716313186845, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'developer': 'mahadi'}>

In [8]:
def read_datafram(filelink):
    df=pd.read_parquet(filelink)

    df.lpep_dropoff_datetime=pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime=pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df=df[(df.duration >=1) & (df.duration <=60 )]

    categorical= ['PULocationID', 'DOLocationID']

    df[categorical]=df[categorical].astype(str)
    
    return df

In [9]:
df_train=read_datafram('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val=read_datafram('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
df_val

In [13]:
target = 'duration'
y_train = df_train[target].values

y_val= df_val[target].values

In [14]:
categorical= ['PU_DO'] #['PULocationID', 'DOLocationID']
numerical =['trip_distance']
dv=DictVectorizer()

train_dicts=df_train[categorical + numerical].to_dict(orient='records')
X_train= dv.fit_transform(train_dicts)


lr = LinearRegression()
lr.fit(X_train, y_train)

val_dicts=df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred=lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715209663881

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [25]:
with mlflow.start_run():
    mlflow.set_experiment_tag("developer", "mahadi")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
    


/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params= params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse=mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status':STATUS_OK}
            

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result= fmin(
    fn= objective,
    space=search_space,
    algo= tpe.suggest,
    max_evals=50,
    trials=Trials())

[09:32:34] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.                              
[0]	validation-rmse:11.52286                                                                                                                        
[1]	validation-rmse:8.13150                                                                                                                         
[2]	validation-rmse:7.08626                                                                                                                         
[3]	validation-rmse:6.76202                                                                                                                         
[4]	validation-rmse:6.63801                                                                                                                         
[5]	validation-rmse:6.58573                                                                               

[53]	validation-rmse:6.44648                                                                                                                        
[54]	validation-rmse:6.44476                                                                                                                        
[55]	validation-rmse:6.44223                                                                                                                        
[56]	validation-rmse:6.44339                                                                                                                        
[57]	validation-rmse:6.44371                                                                                                                        
[58]	validation-rmse:6.44364                                                                                                                        
[59]	validation-rmse:6.44374                                                                              

[107]	validation-rmse:6.44987                                                                                                                       
[108]	validation-rmse:6.44937                                                                                                                       
[109]	validation-rmse:6.44957                                                                                                                       
[110]	validation-rmse:6.44933                                                                                                                       
[111]	validation-rmse:6.44974                                                                                                                       
[112]	validation-rmse:6.44961                                                                                                                       
[113]	validation-rmse:6.44914                                                                             

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:6.78625                                                                                                                         
[1]	validation-rmse:6.69070                                                                                                                         
[2]	validation-rmse:6.66898                                                                                                                         
[3]	validation-rmse:6.67030                                                                                                                         
[4]	validation-rmse:6.66356                                                                                                                         
[5]	validation-rmse:6.65742                                                                                                                         
[6]	validation-rmse:6.64762                                                                               

...

[540]	validation-rmse:6.43913                                                                                                                       
[541]	validation-rmse:6.43896                                                                                                                       
[542]	validation-rmse:6.43856                                                                                                                       
[543]	validation-rmse:6.43843                                                                                                                       
[544]	validation-rmse:6.43808                                                                                                                       
[545]	validation-rmse:6.43788                                                                                                                       
[546]	validation-rmse:6.43761                                                                             

[594]	validation-rmse:6.42930                                                                                                                       
[595]	validation-rmse:6.42908                                                                                                                       
[596]	validation-rmse:6.42889                                                                                                                       
[597]	validation-rmse:6.42872                                                                                                                       
[598]	validation-rmse:6.42815                                                                                                                       
[599]	validation-rmse:6.42809                                                                                                                       
[600]	validation-rmse:6.42765                                                                             

[648]	validation-rmse:6.41969                                                                                                                       
[649]	validation-rmse:6.41951                                                                                                                       
[650]	validation-rmse:6.41908                                                                                                                       
[651]	validation-rmse:6.41893                                                                                                                       
[652]	validation-rmse:6.41858                                                                                                                       
[653]	validation-rmse:6.41854                                                                                                                       
[654]	validation-rmse:6.41851                                                                             

[702]	validation-rmse:6.41114                                                                                                                       
[703]	validation-rmse:6.41115                                                                                                                       
[704]	validation-rmse:6.41095                                                                                                                       
[705]	validation-rmse:6.41086                                                                                                                       
[706]	validation-rmse:6.41064                                                                                                                       
[707]	validation-rmse:6.41039                                                                                                                       
[708]	validation-rmse:6.41025                                                                             

KeyboardInterrupt: 

In [21]:
params= {
    'learning_rate' :0.09722776815289765,
     'max_depth':32,
     'min_child_weight':1.5441518319973766,
     'objective': 'reg:linear',
     'reg_alpha':0.1543662528469625,
     'reg_lambda':0.006730178189027488,
     'seed':42
}

mlflow.xgboost.autolog()

booster = xgb.train(
        params= params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )


2024/05/23 09:45:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '41221394451b4a1ea759bec9dfa72c91', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/05/23 09:45:17 WARNING mlflow.xgboost: Unable to log dataset information to MLflow Tracking.XGBoost version must be >= 1.7.0
2024/05/23 09:45:17 WARNING mlflow.xgboost: Unable to log dataset information to MLflow Tracking.XGBoost version must be >= 1.7.0


[09:45:17] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.45910
[1]	validation-rmse:17.91060
[2]	validation-rmse:16.53042
[3]	validation-rmse:15.30172
[4]	validation-rmse:14.21191
[5]	validation-rmse:13.24422
[6]	validation-rmse:12.38967
[7]	validation-rmse:11.63506
[8]	validation-rmse:10.97256
[9]	validation-rmse:10.39135
[10]	validation-rmse:9.88202
[11]	validation-rmse:9.43824
[12]	validation-rmse:9.05161
[13]	validation-rmse:8.71621
[14]	validation-rmse:8.42521
[15]	validation-rmse:8.17313
[16]	validation-rmse:7.95513
[17]	validation-rmse:7.76705
[18]	validation-rmse:7.60241
[19]	validation-rmse:7.46269
[20]	validation-rmse:7.34058
[21]	validation-rmse:7.23483
[22]	validation-rmse:7.14274
[23]	validation-rmse:7.06453
[24]	validation-rmse:6.99617
[25]	validation-rmse:6.93633
[26]	validation-rmse:6.88458
[27]	validation-rmse:6.83932
[28]	validation-rmse:6.79906
[29]	validation-rmse:6.76415
[30]	valida

[273]	validation-rmse:6.38172
[274]	validation-rmse:6.38132
[275]	validation-rmse:6.38125
[276]	validation-rmse:6.38090
[277]	validation-rmse:6.38080
[278]	validation-rmse:6.38053
[279]	validation-rmse:6.38052
[280]	validation-rmse:6.38021
[281]	validation-rmse:6.37981
[282]	validation-rmse:6.37971
[283]	validation-rmse:6.37952
[284]	validation-rmse:6.37935
[285]	validation-rmse:6.37878
[286]	validation-rmse:6.37860
[287]	validation-rmse:6.37846
[288]	validation-rmse:6.37843
[289]	validation-rmse:6.37790
[290]	validation-rmse:6.37777
[291]	validation-rmse:6.37752
[292]	validation-rmse:6.37729
[293]	validation-rmse:6.37705
[294]	validation-rmse:6.37660
[295]	validation-rmse:6.37633
[296]	validation-rmse:6.37607
[297]	validation-rmse:6.37573
[298]	validation-rmse:6.37538
[299]	validation-rmse:6.37512
[300]	validation-rmse:6.37524
[301]	validation-rmse:6.37502
[302]	validation-rmse:6.37491
[303]	validation-rmse:6.37471
[304]	validation-rmse:6.37473
[305]	validation-rmse:6.37461
[306]	vali

[547]	validation-rmse:6.33551
[548]	validation-rmse:6.33527
[549]	validation-rmse:6.33531
[550]	validation-rmse:6.33499
[551]	validation-rmse:6.33488
[552]	validation-rmse:6.33473
[553]	validation-rmse:6.33463
[554]	validation-rmse:6.33448
[555]	validation-rmse:6.33457
[556]	validation-rmse:6.33461
[557]	validation-rmse:6.33452
[558]	validation-rmse:6.33438
[559]	validation-rmse:6.33435
[560]	validation-rmse:6.33437
[561]	validation-rmse:6.33430
[562]	validation-rmse:6.33419
[563]	validation-rmse:6.33407
[564]	validation-rmse:6.33384
[565]	validation-rmse:6.33369
[566]	validation-rmse:6.33348
[567]	validation-rmse:6.33339
[568]	validation-rmse:6.33337
[569]	validation-rmse:6.33317
[570]	validation-rmse:6.33298
[571]	validation-rmse:6.33273
[572]	validation-rmse:6.33266
[573]	validation-rmse:6.33233
[574]	validation-rmse:6.33232
[575]	validation-rmse:6.33234
[576]	validation-rmse:6.33223
[577]	validation-rmse:6.33212
[578]	validation-rmse:6.33195
[579]	validation-rmse:6.33170
[580]	vali

[821]	validation-rmse:6.31147
[822]	validation-rmse:6.31152
[823]	validation-rmse:6.31119
[824]	validation-rmse:6.31121
[825]	validation-rmse:6.31097
[826]	validation-rmse:6.31092
[827]	validation-rmse:6.31081
[828]	validation-rmse:6.31085
[829]	validation-rmse:6.31089
[830]	validation-rmse:6.31084
[831]	validation-rmse:6.31089
[832]	validation-rmse:6.31096
[833]	validation-rmse:6.31095
[834]	validation-rmse:6.31089
[835]	validation-rmse:6.31078
[836]	validation-rmse:6.31077
[837]	validation-rmse:6.31074
[838]	validation-rmse:6.31083
[839]	validation-rmse:6.31074
[840]	validation-rmse:6.31075
[841]	validation-rmse:6.31076
[842]	validation-rmse:6.31104
[843]	validation-rmse:6.31097
[844]	validation-rmse:6.31097
[845]	validation-rmse:6.31114
[846]	validation-rmse:6.31110
[847]	validation-rmse:6.31102
[848]	validation-rmse:6.31101
[849]	validation-rmse:6.31096
[850]	validation-rmse:6.31086
[851]	validation-rmse:6.31076
[852]	validation-rmse:6.31068
[853]	validation-rmse:6.31058
[854]	vali

2024/05/23 09:47:45 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/23 09:47:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpfs1i3fmd/model, flavor: xgboost). Fall back to return ['xgboost==1.6.0']. Set logging level to DEBUG to see the full traceback. 
2024/05/23 09:47:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [29]:
mlflow.xgboost.autolog(disable=True)

In [30]:
with mlflow.start_run():
    best_params = {
        'learning_rate': 0.09722776815289765,
        'max_depth': 32,
        'min_child_weight': 1.5441518319973766,
        'objective': 'reg:linear',
        'reg_alpha': 0.1543662528469625,
        'reg_lambda': 0.006730178189027488,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv,f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[10:37:59] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.45910
[1]	validation-rmse:17.91060
[2]	validation-rmse:16.53042
[3]	validation-rmse:15.30172
[4]	validation-rmse:14.21191
[5]	validation-rmse:13.24422
[6]	validation-rmse:12.38967
[7]	validation-rmse:11.63506
[8]	validation-rmse:10.97256
[9]	validation-rmse:10.39135
[10]	validation-rmse:9.88202
[11]	validation-rmse:9.43824
[12]	validation-rmse:9.05161
[13]	validation-rmse:8.71621
[14]	validation-rmse:8.42521
[15]	validation-rmse:8.17313
[16]	validation-rmse:7.95513
[17]	validation-rmse:7.76705
[18]	validation-rmse:7.60241
[19]	validation-rmse:7.46269
[20]	validation-rmse:7.34058
[21]	validation-rmse:7.23483
[22]	validation-rmse:7.14274
[23]	validation-rmse:7.06453
[24]	validation-rmse:6.99617
[25]	validation-rmse:6.93633
[26]	validation-rmse:6.88458
[27]	validation-rmse:6.83932
[28]	validation-rmse:6.79906
[29]	validation-rmse:6.76415
[30]	valida

[273]	validation-rmse:6.38172
[274]	validation-rmse:6.38132
[275]	validation-rmse:6.38125
[276]	validation-rmse:6.38090
[277]	validation-rmse:6.38080
[278]	validation-rmse:6.38053
[279]	validation-rmse:6.38052
[280]	validation-rmse:6.38021
[281]	validation-rmse:6.37981
[282]	validation-rmse:6.37971
[283]	validation-rmse:6.37952
[284]	validation-rmse:6.37935
[285]	validation-rmse:6.37878
[286]	validation-rmse:6.37860
[287]	validation-rmse:6.37846
[288]	validation-rmse:6.37843
[289]	validation-rmse:6.37790
[290]	validation-rmse:6.37777
[291]	validation-rmse:6.37752
[292]	validation-rmse:6.37729
[293]	validation-rmse:6.37705
[294]	validation-rmse:6.37660
[295]	validation-rmse:6.37633
[296]	validation-rmse:6.37607
[297]	validation-rmse:6.37573
[298]	validation-rmse:6.37538
[299]	validation-rmse:6.37512
[300]	validation-rmse:6.37524
[301]	validation-rmse:6.37502
[302]	validation-rmse:6.37491
[303]	validation-rmse:6.37471
[304]	validation-rmse:6.37473
[305]	validation-rmse:6.37461
[306]	vali

[547]	validation-rmse:6.33551
[548]	validation-rmse:6.33527
[549]	validation-rmse:6.33531
[550]	validation-rmse:6.33499
[551]	validation-rmse:6.33488
[552]	validation-rmse:6.33473
[553]	validation-rmse:6.33463
[554]	validation-rmse:6.33448
[555]	validation-rmse:6.33457
[556]	validation-rmse:6.33461
[557]	validation-rmse:6.33452
[558]	validation-rmse:6.33438
[559]	validation-rmse:6.33435
[560]	validation-rmse:6.33437
[561]	validation-rmse:6.33430
[562]	validation-rmse:6.33419
[563]	validation-rmse:6.33407
[564]	validation-rmse:6.33384
[565]	validation-rmse:6.33369
[566]	validation-rmse:6.33348
[567]	validation-rmse:6.33339
[568]	validation-rmse:6.33337
[569]	validation-rmse:6.33317
[570]	validation-rmse:6.33298
[571]	validation-rmse:6.33273
[572]	validation-rmse:6.33266
[573]	validation-rmse:6.33233
[574]	validation-rmse:6.33232
[575]	validation-rmse:6.33234
[576]	validation-rmse:6.33223
[577]	validation-rmse:6.33212
[578]	validation-rmse:6.33195
[579]	validation-rmse:6.33170
[580]	vali

[821]	validation-rmse:6.31147
[822]	validation-rmse:6.31152
[823]	validation-rmse:6.31119
[824]	validation-rmse:6.31121
[825]	validation-rmse:6.31097
[826]	validation-rmse:6.31092
[827]	validation-rmse:6.31081
[828]	validation-rmse:6.31085
[829]	validation-rmse:6.31089
[830]	validation-rmse:6.31084
[831]	validation-rmse:6.31089
[832]	validation-rmse:6.31096
[833]	validation-rmse:6.31095
[834]	validation-rmse:6.31089
[835]	validation-rmse:6.31078
[836]	validation-rmse:6.31077
[837]	validation-rmse:6.31074
[838]	validation-rmse:6.31083
[839]	validation-rmse:6.31074
[840]	validation-rmse:6.31075
[841]	validation-rmse:6.31076
[842]	validation-rmse:6.31104
[843]	validation-rmse:6.31097
[844]	validation-rmse:6.31097
[845]	validation-rmse:6.31114
[846]	validation-rmse:6.31110
[847]	validation-rmse:6.31102
[848]	validation-rmse:6.31101
[849]	validation-rmse:6.31096
[850]	validation-rmse:6.31086
[851]	validation-rmse:6.31076
[852]	validation-rmse:6.31068
[853]	validation-rmse:6.31058
[854]	vali

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [31]:

logged_model = 'runs:/b58a01ff18cf4073a13132a1b9f3e551/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


[10:43:37] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [32]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: b58a01ff18cf4073a13132a1b9f3e551

In [33]:
xgboost_model= mlflow.xgboost.load_model(logged_model)

[10:46:09] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [34]:
xgboost_model

In [35]:
y_pred=xgboost_model.predict(valid)

In [36]:
y_pred[:10]

array([14.922333 ,  7.1477666, 13.221982 , 24.335226 ,  9.087692 ,
       17.133648 , 11.546219 ,  8.991295 ,  8.963977 , 20.414936 ],
      dtype=float32)